In [199]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import time
import numpy as np

In [200]:
derecho_dispatch = pd.read_csv('../data/Computer Aided Dispatch Data_Derecho Incidents 050320-050420 - Copy2.csv')

In [201]:
zipcodes = gpd.read_file('../data/zipcodes.geojson')

In [202]:
derecho_dispatch['LandUseType'].value_counts()

Residential                438
Other/Unknown              130
Commercial                  37
Public - Non-Commercial     22
Name: LandUseType, dtype: int64

In [203]:
conditions = [
    derecho_dispatch['LandUseType'] == 'Residential',
    derecho_dispatch['LandUseType'] == 'Commercial',
    derecho_dispatch['LandUseType'] == 'Public - Non-Commercial',
    derecho_dispatch['LandUseType'] == 'Other/Unknown'
    ]

values = ['red', 'orange', 'blue', 'green']

derecho_dispatch['color'] = np.select(conditions, values)

In [204]:
derecho_dispatch['color'].value_counts()

red       438
green     130
orange     37
blue       22
Name: color, dtype: int64

In [205]:
derecho_dispatch['geometry'] = derecho_dispatch.apply(lambda x: Point((float(x.Longitude1), float(x.Latitude1))), axis=1)

In [206]:
derecho_geo = gpd.GeoDataFrame(derecho_dispatch, crs=zipcodes.crs, geometry = derecho_dispatch['geometry'])

In [207]:
zipcodes = zipcodes[['zip', 'po_name', 'geometry']]

In [208]:
derecho_geo = gpd.sjoin(derecho_geo, zipcodes, op = 'within')

In [209]:
polygon37213 = zipcodes.loc[zipcodes['zip'] == '37213']

In [210]:
polygon37213.geometry.centroid

<ipython-input-210-3a0394a5dad1>:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  polygon37213.geometry.centroid


31    POINT (-86.76851 36.16713)
dtype: geometry

In [211]:
center = polygon37213.geometry.centroid[31]
print(center)

POINT (-86.76850874477554 36.16713345170994)


<ipython-input-211-8954b78ae4d7>:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = polygon37213.geometry.centroid[31]


In [212]:
area_center = [center.y, center.x]

In [213]:
print(area_center)

[36.167133451709944, -86.76850874477554]


In [214]:
map_37213 = folium.Map(location = area_center, zoom_start = 12)


In [216]:
for row_index, row_values in derecho_geo.iterrows():
    loc = [row_values['Latitude1'], row_values['Longitude1']]
    marker = folium.CircleMarker(
        location = loc,
        radius=5,
        fill=True,
        color=row_values['color'],
        popup = row_values['IncidentTypeDescription1'])
#        icon=folium.Icon(color=row_values['color'], icon_color=row_values['color']))
    
    marker.add_to(map_37213)
    
map_37213.save('../maps/derecho_map_37213.html')
map_37213

In [ ]:
folium.Map?

cluster_map_37213 = folium.Map(location = area_center, zoom_start = 12)

marker_cluster = MarkerCluster().add_to(cluster_map_37213)

for row_index, row_values in derecho_geo.iterrows():
    loc = [row_values['Latitude1'], row_values['Longitude1']]
    pop = str(row_values[['IncidentDate', 'IncidentTypeDescription1']])
    marker = folium.Marker(
        location = loc, 
        popup = pop) 
    
    marker.add_to(marker_cluster)
    
cluster_map_37213.save('../maps/derecho_cluster_37213.html')
cluster_map_37213
